In [1]:
!pip install openai langchain_openai langchain ragas==0.0.22 pypdf chromadb

In [2]:
# 设置OpenAI KEY环境变量
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI() # 默认是gpt-3.5-turbo

In [4]:
## 切换到当前目录，以便更方便加载文本数据(该步骤只适用于Google Colab)
from google.colab import drive
drive.mount('/content/drive')

import os
path = "/content/drive/My Drive/Colab/RAG/"
os.chdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 通过串联Prompt和llm来构建基本Chain

In [5]:
## 1. 文档加载
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("如何向ChatGPT提问以获得高质量答案：提示技巧工程完全指南.2023.pdf")
pages = loader.load_and_split()

## 2. 文档切分
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)
paragraphs = []
for page in pages:
    paragraphs.extend(text_splitter.create_documents([page.page_content]))

## 3. 文档向量化，向量数据库存储
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(paragraphs, OpenAIEmbeddings())

## 4. 向量检索
retriever = db.as_retriever()
docs = retriever.get_relevant_documents("什么是角色提示？")
for doc in docs:
    print(f"{doc.page_content}\n-------\n")

## 5. 组装Prompt模板
prompt_template = """
你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。

已知信息:
{info}

用户问：
{question}

请用中文回答用户问题。
"""

from langchain.prompts import PromptTemplate
template = PromptTemplate.from_template(prompt_template)
# prompt = template.format(info=docs[0].page_content, question='什么是角色提示？')

## 6. 执行chain
chain = template | llm
response = chain.invoke({"info": docs[0].page_content, "question": "什么是角色提示？"}) ## 给template的输入，多个变量，invoke以字典形式
print(response.content)

第三章：角色提示  
角色提示技术是通过为 ChatGPT 指定一个特定的角色来引导其输出的一种方式。这种技术对于生成针对特定
上下文或受众的文本非常有用。
要使用角色提示技术，您需要为模型提供一个清晰具体的角色。
例如，如果您正在生成客户服务回复，您可以提供一个角色，如 “ 客户服务代表 ” 。
提示公式： “ 作为 [ 角色 ] 生成 [ 任务 ]” 
示例：  
生成客户服务回复：
-------

第十六章：对话提示  
对话提示是一种技术，允许模型生成模拟两个或更多实体之间对话的文本。通过为模型提供一个上下文和一
组角色或实体，以及它们的角色和背景，并要求模型在它们之间生成对话。
因此，应为模型提供上下文和一组角色或实体，以及它们的角色和背景。还应向模型提供有关所需输出的信
息，例如对话或交谈的类型以及任何特定的要求或限制。
提示示例及其公式：  
示例 1 ：对话生成
-------

第十二章：可解释的软提示  
可解释的软提示是一种技术，可以在提供一定的灵活性的同时控制模型生成的文本。它通过提供一组受控输
入和关于所需输出的附加信息来实现。这种技术可以生成更具解释性和可控性的生成文本。
提示示例及其公式：
示例 1 ：文本生成  
任务：生成一个故事  
指令：故事应基于一组给定的角色和特定的主题
-------

这就是  Prompt 工程的作用，通过提供清晰而具体的指令，您可以引导模型的输出并确保其相关。
Prompt 公式是提示的特定格式，通常由三个主要元素组成：  
任务：对提示要求模型生成的内容进行清晰而简洁的陈述。  
指令：在生成文本时模型应遵循的指令。  
角色：模型在生成文本时应扮演的角色。
-------

角色提示是一种通过为ChatGPT指定一个特定角色来引导其输出的技术。通过给模型提供清晰具体的角色，可以帮助生成针对特定上下文或受众的文本。例如，如果您想生成客户服务回复，您可以提供一个角色，如“客户服务代表”。


# 将retriever加进来

In [6]:
# Prompt模板需要的是检索得到的文档块和用户提问。
# retriver的输入是用户query。而用户query也需要跨过retriver，直接放到prompt中。

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
setup_and_retrieval = RunnableParallel(
    {"info": retriever, "question": RunnablePassthrough()}
)

## 6. 执行chain
chain = setup_and_retrieval | template | llm
response = chain.invoke("什么是角色提示？") ## 给retriver的输入，以字符串形式
print(response.content)

角色提示是一种技术，通过为ChatGPT指定一个特定的角色来引导其输出的方式。这种技术对于生成针对特定上下文或受众的文本非常有用。在使用角色提示技术时，您需要为模型提供一个清晰具体的角色，例如客户服务代表。


首先应该重点关注下chain.invoke的输入形式的变化，一个是字典，一个是字符串。

然后，主要是setup_and_retrieval比较难以理解，RunnableParallel和RunnablePassthrough都是新词儿~

RunnablePassthrough是LangChain框架中的一个组件，它允许将输入数据不经修改地传递给下一个步骤，这通常与RunnableParallel一起使用。所以，setup_and_retrieval的意思：

info字段接收retriver的输出

question接收用户的输入，将用户的输入不经修改地传递过来。

这样，我们就把RAG的流程串了起来（前面向量数据库的创建和数据灌入是离线步骤，与这个完全分离开的，不用放到本次的chain里面）。

In [7]:
from langchain.chains import RetrievalQA

# ！！！！！主要应用点：RetrievalQA构建的qa_chain的返回结果
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True,
)
question = "什么是角色提示？"
result = qa_chain({"query": question})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


关键点：使用RetrievalQA去获取结果，因为使用RetrievalQA去获取的结果中包含key：query, result, source_documents，这是LangChain集成的Ragas直接需要的，不用自己再组装数据结构了：

上述result中包含query, result, source_documents字段，这三个字段可以直接用来评估出  context_relevancy， faithfulness， answer_relevancy 三个指标，context_recall无法评估

要想评估 context_recall 指标，==需要人工添加预期的答案==，并添加到 result 的key="ground_truths" 的字段，例如下面的代码

In [8]:
# result_with_truth = result
# result_with_truth["ground_truths"] = "XXXXXXXXXXXX"

引入Ragas封装：RagasEvaluatorChain

引入Ragas评估指标

构造评估的chain，需传入构造的chain的评估指标类型

将上面RAG的结果传入这个评估chain，获得评估结果

In [12]:
## 基于 RAGAS 进行评估
from ragas.metrics import faithfulness, answer_relevancy, context_relevancy, context_recall
from ragas.langchain import RagasEvaluatorChain

# make eval chains
eval_chains = {
    m.name: RagasEvaluatorChain(metric=m)
    for m in [faithfulness, answer_relevancy, context_relevancy] # context_recall暂时先不评估
}

## 评估 RAG 的回答
for name, eval_chain in eval_chains.items():
    score_name = f"{name}_score"
    print(f"{score_name}: {eval_chain(result)[score_name]}")

faithfulness_score: 1.0
answer_relevancy_score: 0.9606677413791962
context_relevancy_score: 0.4375
